<a href="https://colab.research.google.com/github/kaluznys/uczenie_maszynowe_UW/blob/main/praca_domowa6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework Assignment: Understanding Splitting Criteria in CART for Regression**
---------------------

In this assignment, you will explore three common formulations of the splitting criterion used in **CART (Classification and Regression Trees)** for **regression problems**:

1. **Local RSS Minimization**  
2. **RSS Gain Maximization**  
3. **Total RSS Minimization**

You will investigate whether any of these criteria are equivalent, and you will design an experiment to determine which criterion is actually employed in a standard implementation such as **scikit-learn’s DecisionTreeRegressor**.



## **The Problem**

Many treatments of CART for regression describe the split selection process in different ways. Below are three frequently cited formulations. Suppose we have a dataset with features $X$ and target $y$, and we seek to choose a feature $X_j$ and a threshold $t$ to split the data into two regions $R_1(X_j, t)$ and $R_2(X_j, t)$. Denote by $\bar{y}_{R_m}$ the mean of targets within region $R_m$.

1. **Local RSS Minimization**  
   We select the feature and threshold that minimize the **sum of squared errors** in the two resulting child nodes:
   $$
   (X_j^*, t^*) = \arg\min_{X_j, t} \sum_{m=1}^{2} \sum_{i : x_i \in R_m(X_j, t)} (y_i - \bar{y}_{R_m})^2.
   $$

2. **RSS Gain Maximization**  

   It is also a local method, looking only at a parent and two child nodes.

   We select the feature and threshold that maximize the **reduction** in RSS, computed by subtracting the RSS of the two child nodes from the RSS in the parent node:
   $$
   (X_j^*, t^*) = \arg\max_{X_j, t} \Bigl\{
   \underbrace{\sum_{i : x_i \in \text{Parent}} (y_i - \bar{y})^2}_{\text{Parent RSS}}
   \;-\;
   \underbrace{\sum_{m=1}^{2} \sum_{i : x_i \in R_m(X_j, t)} (y_i - \bar{y}_{R_m})^2}_{\text{Children RSS}}
   \Bigr\}.
   $$

3. **Total RSS Minimization**  
   For a dataset $\{(x_i, y_i)\}_{i=1}^N$ with features $X$ and target $y$, let $T$ be the current tree.

   For any split on feature $X_j$ at threshold $t$, define $T(X_j, t)$ as the new tree obtained by splitting one leaf of $T$ into two leaves $R_1(X_j, t)$ and $R_2(X_j, t)$.
   
   Let $\mathrm{Leaves}(T(X_j, t))$ be the set of all leaf indices in this new tree. For each leaf $m \in \mathrm{Leaves}(T(X_j, t))$, define:
   $$
   R_m = \{\, i \,\mid\, x_i \text{ ends in leaf } m\}.
   $$

   $R_m$ set collects all data indices $i$ whose feature vector $x_i$ is classified into the leaf node $m$ when passed through the tree $T(X_j,t)$. In other words, each leaf node $m$ in $T(X_j, t)$ corresponds to a unique path of splits, and any data point $x_i$ that follows that path is assigned to the leaf $m$; hence, it belongs to $R_m$.

   $R_m$ sets for all leafs $m \in \mathrm{Leaves}(T(X_j, t))$ define a partition of all indices.

   Then the objective of **minimizing total Residual Sum of Squares (total RSS)** is stated as:
   $$
   (X_j^*, t^*) = \arg\min_{(X_j, t)} \sum_{m \in \mathrm{Leaves}(T(X_j, t))}
   \sum_{i \in R_m} \Bigl(y_i - \overline{y}_{R_m}\Bigr)^2,
   $$
   where
   $$
   \overline{y}_{R_m} = \frac{1}{\lvert R_m \rvert}
   \sum_{i \in R_m} y_i
   $$
   is the mean response in leaf $m$.


## **Research Questions**

1. **Equivalence Analysis**  
   Determine whether the above formulations are equivalent or if they can yield different split choices. Specifically:
   - Are *local RSS minimization* and *RSS gain maximization* equivalent?
   - Does *total RSS minimization* coincide with either of these two, or is it distinct?
   
2. **Empirical Experiment**  
   Design and conduct a Python experiment to determine which of these formulations is implemented in `scikit-learn` in `DecisionTreeRegressor`. Present numerical results and plots to support your conclusion.


## **Tasks & Deliverables**

1. **Formulation Analysis**  
   - Compare *local RSS minimization*, *RSS gain maximization*, and *total RSS minimization*.
   - If you find that any pair of formulations is equivalent, provide a concise proof.  
   - If you find that they differ, construct a counterexample.

2. **Empirical Verification**  
   - Create a small artificial dataset and train a `DecisionTreeRegressor` from `scikit-learn`.
   - The dataset must be designed in a way that uniquely identifies the formulation used. Provide a short code snippet and a plot or table to support your conclusion.

3. **Report**  
   - Summarize your theoretical insights and empirical findings in a **Colab notebook**.
   - Include the relevant proofs, code, discussion, and conclusions.
   - Place the notebook in your **GitHub repository** for this course, add a link to it in your README.md and add an **“Open in Colab”** badge in the notebook so it can be launched directly.



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
np.random.seed(42)
n_samples=50
ages = np.random.randint(20, 70, size=n_samples)
incomes = np.random.normal(loc=60000, scale=15000, size=n_samples).astype(int)
house_sizes = np.random.normal(loc=50, scale=15, size=n_samples).astype(int)
nr_of_houses = np.random.poisson(lam=1, size=n_samples)
nr_of_houses  = np.ceil(nr_of_houses)+1
locations = np.random.choice(['urban', 'suburban', 'rural'], size=n_samples)

In [ ]:
df = pd.DataFrame({
    'Age': ages,
    'Income': incomes,
    'HouseSize': house_sizes,
    'NrOfHouses': nr_of_houses,
    'Location': locations,
})
df = pd.get_dummies(df, "location")

In [ ]:
house_prices = (
    50000 +
    (ages * -100) +
    (incomes * 0.5) +
    (house_sizes * 4800) +
    (nr_of_houses * 10000) +
    (df.location_rural*-50000) +
    (df.location_suburban*30000)+
    (df.location_urban*-30000) +
    np.random.normal(0, 30000, size=n_samples)
)
df['price'] = house_prices
df.head()

,Age,Income,HouseSize,NrOfHouses,location_rural,location_suburban,location_urban,price
0,58,64365,41,2.0,True,False,False,224728.489667
1,48,50466,27,2.0,False,True,False,313372.529248
2,34,44676,38,1.0,False,True,False,311943.455383
3,62,57573,61,1.0,False,True,False,407515.672469
4,27,51995,46,3.0,False,False,True,307277.314203


In [ ]:
def rss(y):
    if len(y) == 0:
        return 0
    return np.sum((y - np.mean(y)) ** 2)

In [ ]:
class TreeNode:
    def __init__(self, feature=None, value=None, left=None, right=None, prediction=None):
        self.feature = feature
        self.value = value
        self.left = left
        self.right = right
        self.prediction = prediction

In [106]:
def best_split_rss_gain(X, y):
    best_feature, best_value = None, None
    best_rss = float('-inf')

    for feature_idx in range(X.shape[1]):
        values = np.unique(X[:, feature_idx])
        split_candidates = (values[:-1] + values[1:]) / 2
        for val in split_candidates:
          left_mask = X[:, feature_idx] <= val
          right_mask = ~left_mask
          if left_mask.sum() == 0 or right_mask.sum() == 0:
              continue

          y_left, y_right = y[left_mask], y[right_mask]

          rss_gain = rss(y)-(rss(y_left) + rss(y_right))  # Local RSS

          if rss_gain > best_rss and rss_gain>0:
              best_rss = rss_gain
              best_feature = feature_idx
              best_value = val

    return best_feature, best_value, best_rss

In [ ]:
def compute_total_rss(X, y, node):
    if node.prediction is not None:
        return rss(y)

    left_mask = X[:, node.feature] <= node.value
    right_mask = ~left_mask

    rss_left = compute_total_rss(X[left_mask], y[left_mask], node.left)
    rss_right = compute_total_rss(X[right_mask], y[right_mask], node.right)

    return rss_left + rss_right

In [ ]:
def build_tree_rss_gain(X, y, max_depth=3, depth=0):
    if depth >= max_depth:
        return TreeNode(prediction=np.mean(y))

    feature, value, split_rss_gain = best_split_rss_gain(X, y)

    if feature is None:
        return TreeNode(prediction=np.mean(y))

    left_mask = X[:, feature] <= value
    right_mask = ~left_mask

    left = build_tree_rss_gain(X[left_mask], y[left_mask], max_depth, depth + 1)
    right = build_tree_rss_gain(X[right_mask], y[right_mask], max_depth, depth + 1)

    return TreeNode(feature=feature, value=value, left=left, right=right)

In [ ]:
def build_tree_total_rss(X, y, max_depth=2, depth=0):
    if depth >= max_depth:
        return TreeNode(prediction=np.mean(y))

    best_tree = None
    best_rss_total = float('inf')

    for feature_idx in range(X.shape[1]):
        values = np.unique(X[:, feature_idx])
        split_candidates = (values[:-1] + values[1:]) / 2
        for val in values:
            left_mask = X[:, feature_idx] <= val
            right_mask = ~left_mask

            y_left, y_right = y[left_mask], y[right_mask]
            X_left, X_right = X[left_mask], X[right_mask]

            # Skip empty splits
            if len(y_left) == 0 or len(y_right) == 0:
                continue

            # Build subtrees recursively
            left_tree = build_tree_total_rss(X_left, y_left, max_depth, depth + 1)
            right_tree = build_tree_total_rss(X_right, y_right, max_depth, depth + 1)

            # Compute total RSS of whole tree
            rss_left = compute_total_rss(X_left, y_left, left_tree)
            rss_right = compute_total_rss(X_right, y_right, right_tree)
            rss_total = rss_left + rss_right

            if rss_total < best_rss_total:
                best_rss_total = rss_total
                best_tree = TreeNode(
                    feature=feature_idx,
                    value=val,
                    left=left_tree,
                    right=right_tree
                )

    # If no good split, return a leaf
    if best_tree is None:
        return TreeNode(prediction=np.mean(y))

    return best_tree

In [ ]:
def predict_one(x, node):
    while node.prediction is None:
        if x[node.feature] <= node.value:
            node = node.left
        else:
            node = node.right
    return node.prediction

def predict(X, tree):
    return np.array([predict_one(x, tree) for x in X])

In [ ]:
df_nump = df.to_numpy()
X_train, y_train, X_test, y_test = df_nump[30:, :-1], df_nump[30:, -1], df_nump[:20, :-1], df_nump[:20, -1]

In [107]:
gain_std = [0 for i in range(2, 7)]
gain_rmse = [0 for i in range(2, 7)]
for i in range(2, 7):
  tree = build_tree_rss_gain(X_train, y_train, max_depth=i)
  y_pred = predict(X_test, tree)
  print("RSS Gain Tree RMSE:", mean_squared_error(y_test, y_pred)**(1/2))
  gain_rmse[i-2]= round(mean_squared_error(y_test, y_pred)**(1/2),0)
  gain_std[i-2]= round(np.var(y_pred)**(1/2),0)
gain_rmse_std = pd.DataFrame([[gain_rmse[i],gain_std[i]] for i in range(5)])


RSS Gain Tree RMSE: 88834.58492413964
RSS Gain Tree RMSE: 85551.56674672324
RSS Gain Tree RMSE: 85285.06111765814
RSS Gain Tree RMSE: 85367.01823075642
RSS Gain Tree RMSE: 85793.23695538972


In [ ]:
y1 = y_pred

In [108]:
from sklearn.tree import DecisionTreeRegressor
local_std = [0 for i in range(2, 7)]
local_rmse = [0 for i in range(2, 7)]
for i in range(2,7):
  drzewo_local_rss = DecisionTreeRegressor(max_depth=i, random_state=42)
  drzewo_local_rss.fit(X_train, y_train)
  y_pred =drzewo_local_rss.predict(X_test)
  print("RSS local Tree RMSE:", mean_squared_error(y_test, y_pred)**(1/2))
  local_rmse[i-2]= round(mean_squared_error(y_test, y_pred)**(1/2),0)
  local_std[i-2]= round(np.var(y_pred)**(1/2),0)
local_rmse_std = pd.DataFrame([[local_rmse[i],local_std[i]] for i in range(5)])

RSS local Tree RMSE: 88834.58492413964
RSS local Tree RMSE: 79342.1304513447
RSS local Tree RMSE: 78278.78671463106
RSS local Tree RMSE: 79867.46986639452
RSS local Tree RMSE: 78348.33928957352


In [112]:
total_std = [0 for i in range(2, 5)]
total_rmse = [0 for i in range(2, 5)]
for i in range(2, 5):
  tree = build_tree_total_rss(X_train, y_train, max_depth=i)
  y_pred = predict(X_test, tree)
  print("RSS Gain Tree RMSE:", mean_squared_error(y_test, y_pred)**(1/2))
  total_rmse[i-2]= round(mean_squared_error(y_test, y_pred)**(1/2),0)
  total_std[i-2]= round(np.var(y_pred)**(1/2),0)



RSS Gain Tree RMSE: 90981.32622308063
RSS Gain Tree RMSE: 81731.58053781748
RSS Gain Tree RMSE: 84320.72166469411


IndexError: list index out of range

In [114]:
total_rmse_std = pd.DataFrame([[total_rmse[i],total_std[i]] for i in range(3)])
total_rmse_std

,0,1
0,90981.0,83837.0
1,81732.0,89081.0
2,84321.0,97029.0


In [ ]:
y_pred, y1

(array([266729.28613134, 286135.60265724, 223008.68310409, 407678.98861263,
        435756.89154129, 281782.24133204, 359328.14262955, 385503.22472409,
        359328.14262955, 359328.14262955, 385503.22472409, 435756.89154129,
        495412.34988569, 407678.98861263, 379987.81247641, 286135.60265724,
        385503.22472409, 457363.18765336, 140958.46754109, 359328.14262955]),
 array([281782.24133204, 281782.24133204, 140958.46754109, 407678.98861263,
        457363.18765336, 281782.24133204, 359328.14262955, 407678.98861263,
        359328.14262955, 319036.18592359, 385503.22472409, 435756.89154129,
        495412.34988569, 407678.98861263, 359328.14262955, 286135.60265724,
        407678.98861263, 457363.18765336, 223008.68310409, 359328.14262955]))

In [ ]:
tree = build_tree_total_rss(X_train, y_train, max_depth=2)
y_pred = predict(X_test, tree)

from sklearn.metrics import mean_squared_error
print("Total RSS Tree RMSE:", mean_squared_error(y_test, y_pred)**(1/2))

Total RSS Tree RMSE: 0.0


In [ ]:
tree = build_tree_total_rss(X_train, y_train, max_depth=4)
y_pred = predict(X_test, tree)

from sklearn.metrics import mean_squared_error
print("Total RSS Tree RMSE:", mean_squared_error(y_test, y_pred)**(1/2))

Total RSS Tree RMSE: 81529.3976796841


In [ ]:
y_pred_train = predict(X_train, tree)
mean_squared_error(y_train, y_pred_train)**(1/2)

17643.54132112304

In [ ]:
rss(y_pred_train), rss(y_pred)

(np.float64(135678808201.39616), np.float64(79770427635.05501))

In [ ]:
np.unique(df_nump[:, 1])

array([36250, 37377, 38718, 39799, 41023, 43041, 44432, 44676, 44863,
       46194, 46526, 46791, 50466, 50647, 51927, 51995, 53844, 54020,
       56558, 56892, 57334, 57573, 58862, 59087, 59917, 60285, 62016,
       63279, 64365, 64448, 65072, 65840, 68731, 71324, 71605, 71933,
       72521, 73226, 73316, 73414, 73552, 74594, 75637, 76379, 76494,
       77695, 77904, 82431, 101674, 110584], dtype=object)

In [ ]:
rmse_gain= [0 for i in range(2, 5)]
rmse_local_rss= [0 for i in range(2, 5)]
rmse_total_rss= [0 for i in range(2, 5)]
var_gain = [0 for i in range(2, 5)]
var_local_rss = [0 for i in range(2, 5)]
var_total_rss = [0 for i in range(2, 5)]
for i in range(2, 5):
    tree_total = build_tree_total_rss(X_train, y_train, max_depth=i)
    y_pred = predict(X_test, tree_total)
    rmse_total_rss[i-2], var_total_rss[i-2]= mean_squared_error(y_test, y_pred)**(1/2), np.var(y_pred)

    tree_gain = build_tree_rss_gain(X_train, y_train, max_depth=i)
    y_pred_gain = predict(X_test, tree_gain)
    rmse_gain[i-2], var_gain[i-2]= mean_squared_error(y_test, y_pred)**(1/2), np.var(y_pred)

    tree_local_rss = DecisionTreeRegressor(max_depth=4)
    tree_local_rss.fit(X_train, y_train)
    y_pred =tree_local_rss.predict(X_test)
    rmse_local_rss[i-2], var_local_rss[i-2]= mean_squared_error(y_test, y_pred)**(1/2), np.var(y_pred)

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:140: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret / rcount
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:140: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret / rcount


In [ ]:
[rmse_gain, rmse_local_rss, rmse_total_rss, var_gain, var_local_rss, var_total_rss]


[[0.0, 0.22360679774997896, 0.22360679774997896],
 [0.0, 0.0, 0.0],
 [0.0, 0.22360679774997896, 0.22360679774997896],
 [np.float64(0.20999999999999996), np.float64(0.1875), np.float64(0.1875)],
 [np.float64(0.20999999999999996),
  np.float64(0.20999999999999996),
  np.float64(0.20999999999999996)],
 [np.float64(0.20999999999999996), np.float64(0.1875), np.float64(0.1875)]]

If you find that any pair of formulations is equivalent, provide a concise proof.